In [1]:
import pandas as pd
import geopandas as gpd
import os
from datetime import datetime, date, timedelta
import numpy as np

In [2]:
# lodes = pd.read_csv(f"{path}/lodes_combs/lodes_{day}.csv")
lodes = pd.read_csv("/home/rishav/Programs/move_od copy/generated_OD/Davidson_TN_2021-02-01_2021-02-01/lodes_combs/lodes_2021-02-01.csv")
lodes = lodes.rename(
    columns={
        "h_geocode": "home_cbg",
        "w_geocode": "work_cbg",
        "total_jobs": "visits",
        "time_0": "go_time",
        "time_0_str": "go_time_str",
        "time_0_secs": "go_time_secs",
        "time_1": "return_time",
        "time_1_str": "return_time_str",
        "time_1_secs": "return_time_secs",
    }
)

lodes["home_cbg"] = lodes["home_cbg"].astype(str)
# lodes = lodes.merge(chatta[["GEOID"]], left_on="home_cbg", right_on="GEOID")

lodes_go = lodes[
    [
        "home_cbg",
        "work_cbg",
        "home_geom",
        "home_loc_lat",
        "home_loc_lon",
        "work_geom",
        "work_loc_lat",
        "work_loc_lon",
        "visits",
        "go_time",
        "go_time_str",
        "go_time_secs",
    ]
]

lodes_go = lodes_go.rename(
    columns={
        "home_cbg": "pickup_cbg",
        "home_geom": "pickup_geom",
        "home_loc_lat": "pickup_lat",
        "home_loc_lon": "pickup_lon",
        "work_cbg": "dropoff_cbg",
        "work_geom": "dropoff_geom",
        "work_loc_lat": "dropoff_lat",
        "work_loc_lon": "dropoff_lon",
        "go_time": "pickup_time",
        "go_time_str": "pickup_time_str",
        "go_time_secs": "pickup_time_secs",
    }
)

lodes_return = lodes[
    [
        "home_cbg",
        "work_cbg",
        "home_geom",
        "home_loc_lat",
        "home_loc_lon",
        "work_geom",
        "work_loc_lat",
        "work_loc_lon",
        "visits",
        "return_time",
        "return_time_str",
        "return_time_secs",
    ]
]

lodes_return = lodes_return.rename(
    columns={
        "work_cbg": "pickup_cbg",
        "work_geom": "pickup_geom",
        "work_loc_lat": "pickup_lat",
        "work_loc_lon": "pickup_lon",
        "home_cbg": "dropoff_cbg",
        "home_geom": "dropoff_geom",
        "home_loc_lat": "dropoff_lat",
        "home_loc_lon": "dropoff_lon",
        "return_time": "pickup_time",
        "return_time_str": "pickup_time_str",
        "return_time_secs": "pickup_time_secs",
    }
)

lodes_return = lodes_return[lodes_go.columns]

# ADDED TO INDICATE GO AND RETURN TRIPS
lodes_go["direction"] = 0
lodes_return["direction"] = 1

lodes_flattened = pd.concat([lodes_go, lodes_return]).sort_index()

In [3]:
sg = pd.read_csv("/home/rishav/Programs/move_od copy/generated_OD/Davidson_TN_2021-02-01_2021-02-01/safegraph_combs/sg_2021-02-01.csv")
sg = sg.rename(columns={"poi_cbg": "work_cbg"})

sg["home_cbg"] = sg["home_cbg"].astype(str)
# sg = sg.merge(chatta[["GEOID"]], left_on="home_cbg", right_on="GEOID")

sg_go = sg[
    [
        "home_cbg",
        "work_cbg",
        "home_geom",
        "home_loc_lat",
        "home_loc_lon",
        "work_geom",
        "work_loc_lat",
        "work_loc_lon",
        "visits",
        "go_time",
        "go_time_str",
        "go_time_secs",
    ]
]

sg_go = sg_go.rename(
    columns={
        "home_cbg": "pickup_cbg",
        "home_geom": "pickup_geom",
        "home_loc_lat": "pickup_lat",
        "home_loc_lon": "pickup_lon",
        "work_cbg": "dropoff_cbg",
        "work_geom": "dropoff_geom",
        "work_loc_lat": "dropoff_lat",
        "work_loc_lon": "dropoff_lon",
        "go_time": "pickup_time",
        "go_time_str": "pickup_time_str",
        "go_time_secs": "pickup_time_secs",
    }
)

sg_return = sg[
    [
        "home_cbg",
        "work_cbg",
        "home_geom",
        "home_loc_lat",
        "home_loc_lon",
        "work_geom",
        "work_loc_lat",
        "work_loc_lon",
        "visits",
        "return_time",
        "return_time_str",
        "return_time_secs",
    ]
]

sg_return = sg_return.rename(
    columns={
        "work_cbg": "pickup_cbg",
        "work_geom": "pickup_geom",
        "work_loc_lat": "pickup_lat",
        "work_loc_lon": "pickup_lon",
        "home_cbg": "dropoff_cbg",
        "home_geom": "dropoff_geom",
        "home_loc_lat": "dropoff_lat",
        "home_loc_lon": "dropoff_lon",
        "return_time": "pickup_time",
        "return_time_str": "pickup_time_str",
        "return_time_secs": "pickup_time_secs",
    }
)

sg_return = sg_return[sg_go.columns]

# ADDED TO INDICATE GO AND RETURN TRIPS
sg_go["direction"] = 0
sg_return["direction"] = 1

sg_flattened = pd.concat([sg_go, sg_return]).sort_index()

path = '/home/rishav/Programs/move_od copy/generated_OD/Davidson_TN_2021-02-01_2021-02-01/'

union = pd.concat([lodes_flattened, sg_flattened]).sort_index()
union["pickup_time"] = pd.to_datetime(union["pickup_time"])
union["pickup_hour"] = union["pickup_time"].apply(lambda x: x.hour)
union.to_csv(f"{path}/union_all_trips.csv", index=False)

/tmp/ipykernel_339890/1965541367.py:84: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  union["pickup_time"] = pd.to_datetime(union["pickup_time"])


In [12]:
union.head()

,pickup_cbg,dropoff_cbg,pickup_geom,pickup_lat,pickup_lon,dropoff_geom,dropoff_lat,dropoff_lon,visits,pickup_time,pickup_time_str,pickup_time_secs,direction,pickup_hour
0,470370165002,470370165002,POINT (-86.80604379264382 36.140996587440696),36.140997,-86.806044,POINT (-86.80515544660054 36.14759763038929),36.147598,-86.805155,393.0,2023-05-29 10:41:00,10:41,38460.000000,0,10
0,470370165002,470370165002,POINT (-86.80515544660054 36.14759763038929),36.147598,-86.805155,POINT (-86.80604379264382 36.140996587440696),36.140997,-86.806044,393.0,2023-05-29 18:35:00,18:35,66900.000000,1,18
0,470370101031,470370157001,POINT (-86.91335444478325 36.34869181932283),36.348692,-86.913354,POINT (-86.7157277271732 36.10826001490144),36.108260,-86.715728,18.0,2023-05-29 03:35:00,03:35,12900.000000,0,3
0,470370157001,470370101031,POINT (-86.7157277271732 36.10826001490144),36.108260,-86.715728,POINT (-86.91335444478325 36.34869181932283),36.348692,-86.913354,18.0,2023-05-29 06:41:49,06:41,24109.490297,1,6
1,470370165002,470370165002,POINT (-86.80586555015621 36.14297418416197),36.142974,-86.805866,POINT (-86.8055007668664 36.14081082944422),36.140811,-86.805501,393.0,2023-05-29 19:33:00,19:33,70380.000000,1,19
